# Capstone Project - The Battle of Neighborhoods (Week 2)

## Applied DataScience Capstone Project

### Importing Required Libraries

In [28]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

In [29]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')


Data downloaded!


In [30]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [31]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]


{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [32]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)


In [33]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)


In [34]:
neighborhoods.head()


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [35]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [36]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork


In [37]:
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [38]:
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brinx are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Brinx are 40.8466508, -73.8785937.


In [39]:
# create map of Bronx using latitude and longitude values
map_bronx = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(bronx_data['Latitude'], bronx_data['Longitude'], bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bronx)  
    
map_bronx

In [40]:
CLIENT_ID = 'MUPR00XCLBNNHBIYXOBGFKLGYVNE10AIIDDPWXCYFEYVKXWS' # your Foursquare ID
CLIENT_SECRET = 'IV2KZ5A2PSYAJDMIXJRD5V34PYLBGHYQEGS0UE5EDDKHBUQU' # your Foursquare Secret
VERSION = '20190520' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: MUPR00XCLBNNHBIYXOBGFKLGYVNE10AIIDDPWXCYFEYVKXWS
CLIENT_SECRET:IV2KZ5A2PSYAJDMIXJRD5V34PYLBGHYQEGS0UE5EDDKHBUQU


In [41]:
bronx_data.loc[0, 'Neighborhood']


'Wakefield'

In [42]:
neighborhood_latitude = bronx_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = bronx_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = bronx_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


In [43]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=MUPR00XCLBNNHBIYXOBGFKLGYVNE10AIIDDPWXCYFEYVKXWS&client_secret=IV2KZ5A2PSYAJDMIXJRD5V34PYLBGHYQEGS0UE5EDDKHBUQU&v=20190520&ll=40.89470517661,-73.84720052054902&radius=500&limit=100'

In [44]:
results = requests.get(url).json()


In [45]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [46]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lollipops Gelato,Dessert Shop,40.894123,-73.845892
1,Rite Aid,Pharmacy,40.896649,-73.844846
2,Carvel Ice Cream,Ice Cream Shop,40.890487,-73.848568
3,Dunkin',Donut Shop,40.890459,-73.849089
4,Shell,Gas Station,40.894187,-73.845862


In [47]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [49]:

bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


In [50]:
print(bronx_venues.shape)
bronx_venues.head()

(1221, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop
4,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station


In [51]:
bronx_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,33,33,33,33,33,33
Baychester,21,21,21,21,21,21
Bedford Park,37,37,37,37,37,37
Belmont,98,98,98,98,98,98
Bronxdale,14,14,14,14,14,14
Castle Hill,10,10,10,10,10,10
City Island,28,28,28,28,28,28
Claremont Village,20,20,20,20,20,20
Clason Point,10,10,10,10,10,10


In [52]:
bronx_onehot = pd.get_dummies(bronx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bronx_onehot['Neighborhood'] = bronx_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bronx_onehot.columns[-1]] + list(bronx_onehot.columns[:-1])
bronx_onehot = bronx_onehot[fixed_columns]

bronx_onehot.head(3)


,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Donut Shop,Eastern European Restaurant,Electronics Store,Eye Doctor,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Store,Music Venue,Nail Salon,Nightclub,Other Repair Shop,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Recreation Center,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
bronx_grouped = bronx_onehot.groupby('Neighborhood').mean().reset_index()
bronx_grouped

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Donut Shop,Eastern European Restaurant,Electronics Store,Eye Doctor,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Store,Music Venue,Nail Salon,Nightclub,Other Repair Shop,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Recreation Center,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store
0,Allerton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.060606,0.000000,0.000000,0.090909,0.030303,0.030303,0.000000,0.030303,0.000000,0.000,0.000000,0.030303,0.000000,0.030303,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.030303,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.121212,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.030303,0.030303,0.000000,0.000000,0.000000,0.000000,0.060606,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [54]:
bronx_grouped.shape

(52, 176)

In [55]:
num_top_venues = 5

for hood in bronx_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bronx_grouped[bronx_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Allerton----
                venue  freq
0         Pizza Place  0.12
1  Chinese Restaurant  0.09
2       Deli / Bodega  0.09
3         Supermarket  0.06
4      Cosmetics Shop  0.06


----Baychester----
                  venue  freq
0            Donut Shop  0.10
1           Men's Store  0.05
2                  Bank  0.05
3  Gym / Fitness Center  0.05
4     Food & Drink Shop  0.05


----Bedford Park----
                venue  freq
0  Chinese Restaurant  0.14
1               Diner  0.11
2      Sandwich Place  0.05
3  Mexican Restaurant  0.05
4         Bus Station  0.05


----Belmont----
                venue  freq
0  Italian Restaurant  0.19
1       Deli / Bodega  0.08
2         Pizza Place  0.08
3              Bakery  0.05
4          Donut Shop  0.03


----Bronxdale----
                           venue  freq
0             Italian Restaurant  0.07
1  Paper / Office Supplies Store  0.07
2                 Breakfast Spot  0.07
3                    Supermarket  0.07
4             Mexican 

             venue  freq
0     Intersection  0.09
1             Park  0.09
2   Tennis Stadium  0.09
3  Thai Restaurant  0.09
4         Bus Line  0.09


----Throgs Neck----
                 venue  freq
0        Deli / Bodega  0.17
1          Coffee Shop  0.17
2  American Restaurant  0.08
3           Sports Bar  0.08
4            Juice Bar  0.08


----Unionport----
                       venue  freq
0                 Donut Shop  0.09
1  Latin American Restaurant  0.09
2             Ice Cream Shop  0.09
3          Indian Restaurant  0.05
4         Seafood Restaurant  0.05


----University Heights----
                venue  freq
0         Pizza Place  0.16
1          Donut Shop  0.05
2              Bakery  0.05
3          Shoe Store  0.05
4  Chinese Restaurant  0.05


----Van Nest----
                       venue  freq
0                Pizza Place  0.25
1              Deli / Bodega  0.25
2  Middle Eastern Restaurant  0.10
3                 Playground  0.05
4                Coffee Shop  0.0

In [56]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [57]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bronx_grouped['Neighborhood']

for ind in np.arange(bronx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Chinese Restaurant,Cosmetics Shop,Supermarket,Dessert Shop,Bus Station,Spa,Spanish Restaurant,Check Cashing Service
1,Baychester,Donut Shop,Mattress Store,Supermarket,Discount Store,Pizza Place,Convenience Store,Electronics Store,Fast Food Restaurant,Sandwich Place,Bus Station
2,Bedford Park,Chinese Restaurant,Diner,Supermarket,Food Truck,Bus Station,Mexican Restaurant,Sandwich Place,Pizza Place,Deli / Bodega,Pub
3,Belmont,Italian Restaurant,Deli / Bodega,Pizza Place,Bakery,Dessert Shop,Donut Shop,Bank,Coffee Shop,Mexican Restaurant,Eastern European Restaurant
4,Bronxdale,Breakfast Spot,Doctor's Office,Mexican Restaurant,Spanish Restaurant,Supermarket,Garden,Eastern European Restaurant,Chinese Restaurant,Gym,Bank


In [58]:
# set number of clusters
kclusters = 5

bronx_grouped_clustering = bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 2, 4, 1, 1, 4, 4, 4, 4, 4], dtype=int32)

In [59]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bronx_merged = bronx_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bronx_merged = bronx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bronx_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,2,Ice Cream Shop,Laundromat,Pharmacy,Donut Shop,Sandwich Place,Caribbean Restaurant,Gas Station,Dessert Shop,Health & Beauty Service,Flea Market
1,Bronx,Co-op City,40.874294,-73.829939,4,Baseball Field,Bus Station,Pharmacy,Bagel Shop,Mattress Store,Fast Food Restaurant,Liquor Store,Gift Shop,Basketball Court,Grocery Store
2,Bronx,Eastchester,40.887556,-73.827806,4,Caribbean Restaurant,Deli / Bodega,Diner,Bus Station,Seafood Restaurant,Pizza Place,Platform,Metro Station,Cosmetics Shop,Fast Food Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,0,Plaza,Bus Station,River,Women's Store,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
4,Bronx,Riverdale,40.890834,-73.912585,4,Park,Bus Station,Bank,Medical Supply Store,Baseball Field,Gym,Food Truck,Home Service,Plaza,Dive Bar


In [60]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bronx_merged['Latitude'], bronx_merged['Longitude'], bronx_merged['Neighborhood'], bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [61]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 0, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,Plaza,Bus Station,River,Women's Store,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant


In [62]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 1, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Woodlawn,Pub,Deli / Bodega,Pizza Place,Playground,Bar,Italian Restaurant,Food Truck,Moving Target,Donut Shop,Park
10,Pelham Parkway,Pizza Place,Italian Restaurant,Frozen Yogurt Shop,Chinese Restaurant,Coffee Shop,Sandwich Place,Gas Station,Gift Shop,Sushi Restaurant,Bank
27,Throgs Neck,Deli / Bodega,Coffee Shop,Pizza Place,Asian Restaurant,Sports Bar,Liquor Store,American Restaurant,Juice Bar,Bar,Italian Restaurant
31,Van Nest,Deli / Bodega,Pizza Place,Middle Eastern Restaurant,Playground,Supermarket,Donut Shop,Coffee Shop,Caribbean Restaurant,Bus Line,Board Shop
32,Morris Park,Pizza Place,Bakery,Deli / Bodega,Burger Joint,Juice Bar,Sandwich Place,Supermarket,Coffee Shop,Buffet,Bar
33,Belmont,Italian Restaurant,Deli / Bodega,Pizza Place,Bakery,Dessert Shop,Donut Shop,Bank,Coffee Shop,Mexican Restaurant,Eastern European Restaurant
35,North Riverdale,Pizza Place,Deli / Bodega,Italian Restaurant,Bank,Sushi Restaurant,Donut Shop,Pool,Coffee Shop,Chinese Restaurant,Mobile Phone Shop
38,Edgewater Park,Italian Restaurant,Deli / Bodega,Donut Shop,Pizza Place,Bar,Chinese Restaurant,Sports Bar,Coffee Shop,Pub,Juice Bar
49,Bronxdale,Breakfast Spot,Doctor's Office,Mexican Restaurant,Spanish Restaurant,Supermarket,Garden,Eastern European Restaurant,Chinese Restaurant,Gym,Bank


In [63]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 2, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Ice Cream Shop,Laundromat,Pharmacy,Donut Shop,Sandwich Place,Caribbean Restaurant,Gas Station,Dessert Shop,Health & Beauty Service,Flea Market
5,Kingsbridge,Pizza Place,Bar,Mexican Restaurant,Bakery,Supermarket,Sandwich Place,Donut Shop,Spanish Restaurant,Pub,Pharmacy
9,Baychester,Donut Shop,Mattress Store,Supermarket,Discount Store,Pizza Place,Convenience Store,Electronics Store,Fast Food Restaurant,Sandwich Place,Bus Station
13,University Heights,Pizza Place,Fried Chicken Joint,Bakery,Sandwich Place,Fast Food Restaurant,Chinese Restaurant,Clothing Store,Supermarket,Laundromat,Latin American Restaurant
15,Fordham,Fast Food Restaurant,Mobile Phone Shop,Donut Shop,Pizza Place,Shoe Store,Gym / Fitness Center,Bank,Pharmacy,Juice Bar,Spanish Restaurant
17,West Farms,Bus Station,Sandwich Place,Donut Shop,Bank,Metro Station,Bus Stop,Diner,Lounge,Supermarket,Basketball Court
19,Melrose,Pizza Place,Pharmacy,Sandwich Place,Supermarket,Spa,Diner,Martial Arts Dojo,Donut Shop,Supplement Shop,Clothing Store
20,Mott Haven,Pizza Place,Spanish Restaurant,Gym,Donut Shop,Pharmacy,Bakery,Fish & Chips Shop,Peruvian Restaurant,Mobile Phone Shop,Bookstore
21,Port Morris,Cupcake Shop,Latin American Restaurant,Peruvian Restaurant,Distillery,Donut Shop,Restaurant,Music Venue,Brewery,Storage Facility,Furniture / Home Store
22,Longwood,Deli / Bodega,Diner,Donut Shop,Bus Stop,Latin American Restaurant,Sandwich Place,Fast Food Restaurant,Grocery Store,Women's Store,Farmers Market


In [64]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 3, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Williamsbridge,Caribbean Restaurant,Nightclub,Soup Place,Bar,Dance Studio,Dive Bar,Doctor's Office,Food,Flower Shop,Diner


In [65]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 4, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,Baseball Field,Bus Station,Pharmacy,Bagel Shop,Mattress Store,Fast Food Restaurant,Liquor Store,Gift Shop,Basketball Court,Grocery Store
2,Eastchester,Caribbean Restaurant,Deli / Bodega,Diner,Bus Station,Seafood Restaurant,Pizza Place,Platform,Metro Station,Cosmetics Shop,Fast Food Restaurant
4,Riverdale,Park,Bus Station,Bank,Medical Supply Store,Baseball Field,Gym,Food Truck,Home Service,Plaza,Dive Bar
7,Norwood,Pizza Place,Bank,Park,Deli / Bodega,Chinese Restaurant,Pharmacy,Bus Station,Cosmetics Shop,Liquor Store,Supermarket
11,City Island,Harbor / Marina,Seafood Restaurant,Thrift / Vintage Store,Pizza Place,Park,Smoke Shop,History Museum,Pharmacy,Deli / Bodega,Diner
12,Bedford Park,Chinese Restaurant,Diner,Supermarket,Food Truck,Bus Station,Mexican Restaurant,Sandwich Place,Pizza Place,Deli / Bodega,Pub
14,Morris Heights,Pharmacy,Recreation Center,Spanish Restaurant,Bus Station,Moving Target,Check Cashing Service,Grocery Store,Bank,Pizza Place,Playground
16,East Tremont,Pizza Place,Park,Restaurant,Fast Food Restaurant,Breakfast Spot,Shoe Store,Mobile Phone Shop,Café,Lounge,Supermarket
18,High Bridge,Pizza Place,Pharmacy,Sandwich Place,Seafood Restaurant,Food,Latin American Restaurant,Playground,Chinese Restaurant,Donut Shop,Check Cashing Service
25,Soundview,Chinese Restaurant,Grocery Store,Fried Chicken Joint,Latin American Restaurant,Discount Store,Playground,Bus Stop,Bus Station,Food,Burger Joint


### Results
------------------------------------------------------------------------------------------------------------------------------------------------------
We have sucessfully retrived the list of hotels in New York City and able to get the ratings data. The sorting of the data is done as per the ratings and filtered the Top 5 Hotels. The places around those filtered hotels are also listed so that the users can pick the hotels based on there interest. we can able to show this hotels in a map along with the places. This is just a begining analysis where lot more can be done further like food and other facilities provided by the hotels,check-In and check-Out time and lot more.

### Conclusion
------------------------------------------------------------------------------------------------------------------------------------------------------

I would like to conclude the project as the user can get the top 5 hotels listed and can get the visiting places surounded by that hotels which help the users to pick the best hotel as per their requirement and their area of interests.  

#### ThankYou :)
